In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Arial']
matplotlib.rcParams['font.sans-serif'] = ['System Font', 'Verdana', 'Arial']
matplotlib.rcParams['figure.dpi'] = 108

In [ ]:
def sgfit(x, y, va=None, noise=False):
    if va is None:
        # Normal Gaussian fitting
        sum_x = np.sum(x)
        sum_x2 = np.sum(x ** 2)
        sum_x3 = np.sum(x ** 3)
        sum_x4 = np.sum(x ** 4)
        M = np.matrix([[len(x), sum_x, sum_x2], [sum_x, sum_x2, sum_x3], [sum_x2, sum_x3, sum_x4]])
        B = np.matrix([[np.sum(np.log(y))], [np.sum(np.multiply(x, np.log(y)))], [np.sum(np.multiply(x ** 2, np.log(y)))]])
        d = np.linalg.solve(M, B)
        a = d.item(0)
        b = d.item(1)
        c = d.item(2)

        A = np.exp(a - b ** 2 / (4 * c))
        mu = -b / (2 * c)
        if c > 0:
            # Failed
            print('Warning. Unable to estimate sig.')
            sig = 1.0e-7
        else:
            sig = np.sqrt(-1 / (2 * c))
    else:
        # Normalize the x-values to [-pi, pi]
        omega = x / va * np.pi
        
        # Complex-plane representation
        c = np.multiply([max(0, x) for x in y], np.exp(1j * omega))
        
        # Angle of the sum of the vectors
        mu = np.angle(np.sum(c))
        
        # Shift the x-axis
        d = c * np.exp(-1j * mu)
        
        # New x-axis
        x = np.angle(d) / np.pi * va;
        
        # Convert mu back to original span
        mu = mu / np.pi * va

        # Now we estimate
        if noise:
            y2 = y ** 2
            for k in range(10):
                x2 = x ** 2
                x4 = x2 ** 2
                sum_y2 = np.sum(y2)
                sum_x2_y2 = np.sum(x2 * y2)
                sum_x4_y2 = np.sum(x4 * y2)
                # 2 x 2 matrix inversion
                a = sum_y2; b = c = sum_x2_y2; d = sum_x4_y2
                M = 1.0 / (a * d - b * c) * np.array([[d, -b], [-c, a]])
                B = np.array([np.sum(y2 * np.log(y)), np.sum((x2 * y2) * np.log(y))])
                d = np.matmul(M, B)

                Ak = np.exp(d[0])
                sigk = np.sqrt(-1 / (2 * d[1]))
                print('mu = {0:.4f}   sig = {1:.4f}   A = {2:.4f} (k = {3})'.format(mu, sigk, Ak, k))

                y2 = np.exp(d[0] + d[1] * x2)

            A = Ak
            sig = sigk
        else:
            sum_x2 = np.sum(np.abs(x) ** 2)
            sum_x4 = np.sum(np.abs(x) ** 4)
            a = len(x); b = c = sum_x2; d = sum_x4
            M = 1.0 / (a * d - b * c) * np.array([[d, -b], [-c, a]])
            B = np.array([np.sum(np.log(y)), np.sum(np.multiply(x ** 2, np.log(y)))])
            d = np.matmul(M, B)

            A = np.exp(d[0])
            sig = np.sqrt(-1 / (2 * d[1]))
    return A, sig, mu

In [ ]:
def spec(x, va, A, mu, sig, N):
    y = (A * np.exp(-(x - mu) ** 2 / (2 * sig ** 2))
    + A * np.exp(-(x - mu + 2 * va) ** 2 / (2 * sig ** 2))
    + A * np.exp(-(x - mu - 2 * va) ** 2 / (2 * sig ** 2))
    + 0.5 * N)
    return y

In [ ]:
# Some basic parameters
N = 60
va = 15
x = np.arange(0, N) / N * 2 * va - va

A = 0.5
mu = 11
sig = 5
An = 0.1 * A

# Generate a prestine spectrum
y = spec(x, va, A, mu, sig, An)

# Store a copy of the original
yo = y

# Pertube the amplitudes
y = (np.random.rand(N) + 0.5) * y

# Additive noise
n = An * (np.random.rand(N) - 0.5)

# Add noise
y = y + n

# Some threshold to select what data samples to use
th = 0.5 * np.sqrt(np.mean(y ** 2))

# Mask of good samples
mask = y > th

# With Noise Estimate

We can safely assume that the system noise can be esimated to the necessary accuracy.

In [ ]:
ye = y - 0.5 * An

### Using Conventional Fit

A warning will be generated since sig cannot be estimated if the spectrum has a mainlobe that spans across the aliasing limits.

In [ ]:
A1, sig1, mu1 = sgfit(x[mask], ye[mask])
y1s = spec(x, va, A1, mu1, sig1, An)

### Using SGFIT

This is the primary problem SGFIT is designed for. That is, noise can be estimated reasonably well and be removed from the data samples.

In [ ]:
A2, sig2, mu2 = sgfit(x[mask], ye[mask], va)
y2s = spec(x, va, A2, mu2, sig2, An)

# Without Noise Esimate

The noise level is modeled in the iterative SGFIT. In this approach, an iterative algorithm is used so the noise level is actually one of the estimated parameters. About 10 iteraction is usually sufficient. The width `sig` tends to be over-estimated if the number of iterations is limited.

In [ ]:
# A3, sig3, mu3 = sgfit(x[mask], y[mask], va, noise=True)
A3, sig3, mu3 = sgfit(x, y, va, noise=True)
y3s = spec(x, va, A3, mu3, sig3, An)

# Show All Results

In [ ]:
highlight = [0.85, 0.96, 0]

gt = 'mu = {0:.4f}   sig = {1:.4f}   A = {2:.4f} (ground truth)'.format(mu, sig, A)
m1 = 'mu = {0:.4f}   sig = {1:.4f}   A = {2:.4f} (conventional)'.format(mu1, sig1, A1)
m2 = 'mu = {0:.4f}   sig = {1:.4f}   A = {2:.4f} (SGFIT)'.format(mu2, sig2, A2)
m3 = 'mu = {0:.4f}   sig = {1:.4f}   A = {2:.4f} (SGFIT iter)'.format(mu3, sig3, A3)

fig = matplotlib.pyplot.figure(figsize=(8,4))
# fig.suptitle('Conventional Gaussian Fitting vs SGFIT', fontweight='bold')
fig.suptitle('Gaussian Fitting', fontweight='bold')
if not all(mask) and mask[0]:
    e = list(mask).index(False)
    matplotlib.pyplot.plot(x[0:e], y[0:e], color=highlight, linewidth=5)
    e = N - list(mask[::-1]).index(False)
    matplotlib.pyplot.plot(x[e:], y[e:], color=highlight, linewidth=5)
else:
    matplotlib.pyplot.plot(x[mask], y[mask], color=highlight, linewidth=5)
h1, = matplotlib.pyplot.plot(x[mask], y[mask], '.', color=highlight, markersize=20)
h2, = matplotlib.pyplot.plot(x, y, '.', MarkerSize=10, color=[0.85, 0.2, 0])
h3, = matplotlib.pyplot.plot(x, yo, color=[0.9, 0.6, 0])
h4, = matplotlib.pyplot.plot(x, y1s, '--', color=[0.7, 0.2, 0.8])
h5, = matplotlib.pyplot.plot(x, y2s, '-.', color=[0.4, 0.8, 0.0])
h6, = matplotlib.pyplot.plot(x, y3s, '-.', color=[0.3, 0.2, 0.8])
matplotlib.pyplot.grid()
matplotlib.pyplot.ylim((0.0, 2.0 * A))
matplotlib.pyplot.xlabel('Velocity (m/s)')
matplotlib.pyplot.ylabel('Amplitude')
matplotlib.pyplot.legend([h1, h2, h3, h4, h5, h6], ['Used Samples', 'Original Dataset', gt, m1, m2, m3])
# matplotlib.pyplot.legend([h1, h2, h3, h4], ['Used Samples', 'Original Dataset', tstr, m1])

In [ ]:
# fig.savefig('blob/fig2.png')